# **Segmenting and Clustering Neighborhoods in Toronto**

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
3. To create the dataframe:
    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
    - If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
    - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

### Loading packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Obtain the data from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

### Using BeautifulSoup to parse HTML

In [3]:
data = response.text
soup = BeautifulSoup(data,'html.parser')
table = soup.find('table')

### Create the dataframe

In [4]:
df = pd.read_html(str(table))[0]
df.columns = ["PostalCode", "Borough", "Neighborhood"]
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Ignore cells with a borough that is Not assigned

In [5]:
df = df.drop(df[(df.Borough == "Not assigned")].index)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [6]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Rows with the same postal code will be combined into one row with the Neighbourhoods separated with a comma

In [7]:
df = df.groupby(['PostalCode','Borough'], sort = False).agg( ', '.join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Use the .shape method to print the number of rows of your dataframe

In [8]:
df.shape

(103, 3)

# PART 2

### Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

Here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [9]:
url = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo_data = pd.read_csv(url)
df_geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Rename column name to match with df

In [10]:
df_geo_data = df_geo_data.rename(columns = {'Postal Code':'PostalCode'}) 
df_geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge df with the df_geo_data to add 2 news columns

In [11]:
df = pd.merge(df, df_geo_data, on = 'PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
df.shape

(103, 5)

# PART 3

### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

### Filter only "Borough" which contain the word Toronto 

In [13]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop = True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [14]:
toronto_data.shape

(39, 5)

### Import necessary packages

In [16]:
from geopy.geocoders import Nominatim
# !pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import numpy as np

### Use geopy library to get the latitude and longitude values of Toronto

In [17]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
neighborhoods = toronto_data
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top

In [18]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them

In [19]:
CLIENT_ID = 'HIDDEN' # your Foursquare ID
CLIENT_SECRET = 'HIDDEN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500 # define radius

### Send the GET request and examine the results

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the above function on each neighborhood and create a new dataframe called manhattan_venues

In [23]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

### Check the size of the resulting dataframe

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(2886, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,"Regent Park, Harbourfront",43.65426,-79.360636,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,"Regent Park, Harbourfront",43.65426,-79.360636,Japango,43.655268,-79.385165,Sushi Restaurant
3,"Regent Park, Harbourfront",43.65426,-79.360636,Indigo,43.653515,-79.380696,Bookstore
4,"Regent Park, Harbourfront",43.65426,-79.360636,Poke Guys,43.654895,-79.385052,Poke Place


### Analyze Each Neighborhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Vietnamese Restaurant,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,...,Shoe Store,Shopping Mall,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(2886, 53)

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Vietnamese Restaurant,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,...,Shoe Store,Shopping Mall,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store
0,Berczy Park,0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
1,"Brockton, Parkdale Village, Exhibition Place",0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
2,"Business reply mail Processing Centre, South C...",0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
4,Central Bay Street,0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
5,Christie,0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
6,Church and Wellesley,0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
7,"Commerce Court, Victoria Hotel",0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
8,Davisville,0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514
9,Davisville North,0.013514,0.027027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.013514,0.027027,0.027027,0.027027,0.013514


### Print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0      Clothing Store  0.08
1         Coffee Shop  0.07
2  Seafood Restaurant  0.04
3               Hotel  0.03
4               Plaza  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0      Clothing Store  0.08
1         Coffee Shop  0.07
2  Seafood Restaurant  0.04
3               Hotel  0.03
4               Plaza  0.03


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0      Clothing Store  0.08
1         Coffee Shop  0.07
2  Seafood Restaurant  0.04
3               Hotel  0.03
4               Plaza  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0      Clothing Store  0.08
1         Coffee Shop  0.07
2  Seafood Restaurant  0.04
3               Hotel  0.03
4               Plaza  0.03


----Central Bay Street----
            

### Write a function to sort the venues in descending order

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each neighborhood

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse
1,"Brockton, Parkdale Village, Exhibition Place",Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse
2,"Business reply mail Processing Centre, South C...",Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse
3,"CN Tower, King and Spadina, Railway Lands, Har...",Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse
4,Central Bay Street,Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse


### Cluster Neighborhoods

In [33]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

### Run k-means to cluster the neighborhood into 5 clusters

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [38]:
toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Clothing Store,Coffee Shop,Seafood Restaurant,Plaza,Vegetarian / Vegan Restaurant,Restaurant,Cosmetics Shop,Diner,Hotel,Steakhouse


### Visualize the resulting clusters

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters